<a id="top"></a>
<img style="width:40%;max-width:600px" alt="Bluelight AI Logo" href="https://bluelightai.com/" src="https://github.com/BlueLightAI/cobalt-examples/blob/main/assets/blai-logo-light.png?raw=true">

# Cobalt UI Walkthrough

<a href="https://bluelightai.com/contact">Give Feedback</a> | <a href="https://bluelightai.com/">Our Website</a> | <a href="https://bluelightai.com/blog">Our Blog</a> | <a href="https://docs.cobalt.bluelightai.com/">Cobalt Docs</a> | <a href="https://join.slack.com/t/bluelightaicom/shared_invite/zt-2uj0iu5lh-5WgutuwH82RxAOwuq8ptqg">Slack Community</a>

**Tags:** #blai #python #cobaltai #tda #embedding #cobaltui #imageclassification

**Last update:** 2024-12-12 (Created: 2024-11-15)

## Goals:

### What you will see:

- Download the imagenette data (with 10 classes),
- Generate embedding using a Clip model for the data,
- Visualize the embedded results using a powerful visualization tool, 
- Generate insights using the visual analysis, and 
- Select subgroups from the UI and interact with selected subgroup using python code.

### You will learn:

- Introduction to Cobalt and its UI, and
- Application of Cobalt with embedding and classification tasks.

<u>References:</u>
- [Intelligent Search Results for Ecommerce: BluelightAI and Marqo Join Forces](https://bluelightai.com/blog/ecommerce-industry-partnership-with-marqo)
- [Curate Your Datasets with Cobalt for Higher Performing Models](https://bluelightai.com/blog/bluelightai-cobalt-control-your-datasets-with-topological-data-analysis)

## Input

### Install dependencies

First, we install the CLIP model and PyTorch to create embeddings from some sample data. We follow the setup instructions from the OpenAI's [CLIP](https://github.com/openai/CLIP) repository.

In [1]:
# %pip install ftfy packaging regex tqdm torch torchvision
# %pip install git+https://github.com/openai/CLIP.git

Now dowanload the cobalt package from PyPI:

In [2]:
# %pip install cobalt-ai
# %pip install --upgrade cobalt-ai

Lastly, lets download the Imagenette dataset: 

In [3]:
# torchvision.datasets.Imagenette(root=".", split="val", download=True)

This will download data in the current working directory for the notebook. If you want to download it somewhere else, you can specify the root argument with a different path.

### Imports libraries

In [4]:
import os

import clip
import numpy as np
import pandas as pd
import torch
import torchvision
from PIL import Image
from tqdm import tqdm

import cobalt

If you have not registered Cobalt yet, please uncomment and run the following to register:

In [5]:
# cobalt.register_license()

### Device

Lets set up the device to run the CLIP model on. We'll use a GPU if available, otherwise we'll fall back to using the CPU.

In [6]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

device

'mps'

### Data

We use the [Imagenette dataset](https://github.com/fastai/imagenette), a subset of the ImageNet dataset on 10 very different classes. (See the dependencies section above for download command.) 

This data has is split in two parts, training (9,469 images) and validation (3,925 images):

In [7]:
# If this gives an error, please see the dependencies section to download the data
torchvision.datasets.Imagenette(root=".", split="train")

Dataset Imagenette
    Number of datapoints: 9469
    Root location: .

In [8]:
torchvision.datasets.Imagenette(root=".", split="val")

Dataset Imagenette
    Number of datapoints: 3925
    Root location: .

For this introduction to Cobalt UI, we will use smaller validation data:

In [9]:
imagenette_training = torchvision.datasets.Imagenette(root=".", split="val")

Now, let us create a list of file paths for all the images in the dataset in the variable `training_images_paths`, and also note the `training_class_indices` for each of these image:

In [10]:
training_images_root = imagenette_training._image_root

# Create a list of image paths and corresponding target indices.
training_images_paths = []
training_class_indices = []
for c in os.listdir(training_images_root):
    class_path = os.path.join(training_images_root, c)
    image_index = imagenette_training.wnid_to_idx[c]
    for f in os.listdir(class_path):
        training_images_paths.append(os.path.join(class_path, f))
        training_class_indices.append(image_index)

In [11]:
print(f'Total files: {len(training_images_paths)}')

Total files: 3925


In [12]:
training_images_paths[:3]

['imagenette2/val/n03394916/n03394916_32422.JPEG',
 'imagenette2/val/n03394916/n03394916_69132.JPEG',
 'imagenette2/val/n03394916/n03394916_33771.JPEG']

In [13]:
training_class_indices[:3]

[5, 5, 5]

Note that these class indices are in numeric form. 

Imagenette data provides human readable class labels for each of these images:

In [14]:
imagenette_training.classes

[('tench', 'Tinca tinca'),
 ('English springer', 'English springer spaniel'),
 ('cassette player',),
 ('chain saw', 'chainsaw'),
 ('church', 'church building'),
 ('French horn', 'horn'),
 ('garbage truck', 'dustcart'),
 ('gas pump', 'gasoline pump', 'petrol pump', 'island dispenser'),
 ('golf ball',),
 ('parachute', 'chute')]

Now we convert `training_class_indices` to human readable `training_labels`:

In [15]:
index_to_class = [c[0] for c in imagenette_training.classes]
training_labels = [index_to_class[y] for y in training_class_indices]
training_labels[:3]

['French horn', 'French horn', 'French horn']

Lets see how many images are associated with each of these 10 labels:

In [16]:
pd.Series(training_labels).value_counts()

gas pump            419
church              409
golf ball           399
English springer    395
French horn         394
parachute           390
garbage truck       389
tench               387
chain saw           386
cassette player     357
Name: count, dtype: int64

So, roughly 350 to 420 images are there in each of these 10 categories.

Now, we are ready to generate embeddings.

### Embedding Model

In this example, we're using a [CLIP model](https://github.com/openai/CLIP) to analyze the images to generate embeddings or numerical representations of images. 

The "ViT-B/16" model here refers to a Vision Transformer model with an embedding size of 512 dimensions. Given an image or text passage, this model outputs a 512-dimensional vector representing the data. Images or passages with similar content should have embedding vectors with a high cosine similarity.

In [17]:
model, preprocess = clip.load("ViT-B/16", device=device)

In [18]:
{"modelParameterCount": f'{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}',
 "inputResolution" : model.visual.input_resolution,
 "contextLength" : model.context_length,
 "vocabSize": model.vocab_size
}

{'modelParameterCount': '149,620,737',
 'inputResolution': 224,
 'contextLength': 77,
 'vocabSize': 49408}

This specific model is chosen here because it provides a good balance between performance and computational efficiency for many image analysis tasks. It is not the best in terms of accuracy, but allows us to introduce some aspects of Cobalt UI that users will find useful. 

There are many models available in the CLIP model family, each with different architectural configurations and capabilities:

In [19]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

We use some of these models in more advanced examples in later examples.

### Generate Embeddings

Now, for each of these images we generate an embedding, and then we stack all of those embeddings together vertically to create an embedding matrix. 

In [20]:
# Iterate through training_images_paths to build an embedding of shape (N x 512).
# This may take about 1-3 minutes depending on your machine.
embedding_training = []
for p in tqdm(training_images_paths):
    with torch.no_grad():
        image = preprocess(Image.open(p)).unsqueeze(0).to(device)
        image_features = model.encode_image(image)
        embedding_training.append(image_features)

embedding_np_training = [element.cpu().numpy() for element in embedding_training]
embedding_matrix_training = np.concatenate(embedding_np_training)

100%|██████████| 3925/3925 [01:13<00:00, 53.75it/s]


In [21]:
embedding_matrix_training.shape

(3925, 512)

The number of rows in the embedding matrix above is the number of images, and the number of columns is the number of embedding dimensions.

Now we're ready to explore how Cobalt groups the Imagenette dataset based on these CLIP embeddings. 

### Cobalt Workspace and UI

Let's load this metadata into cobalt:
1. *Image labels*: We add image labels so that we can see if the CLIP embeddings are consistent with the target labels of the dataset.
2. *The embedding matrix*: We use `add_embedding_array` to add an embedding. Note that every element in your dataframe needs to have a corresponding row in your embedding.
3. *Raw image file paths*: We're going to need to perform one additional step of `add_media_column` to pass in a list of training_images_paths that should be interpreted as images for Cobalt to display. 

In [22]:
# Step 1. Create a pandas data frame with reference labels
df = pd.DataFrame({"targets": training_labels})
df["targets"] = df["targets"].astype("category")

# Step 2. Convert the data frame to CobaltDataset and add embedding array and image paths.
ds = cobalt.CobaltDataset(df)
ds.add_embedding_array(embedding_matrix_training)
ds.add_media_column(training_images_paths, local_root_path=imagenette_training.root)

# Step 3. Create a workspace with the CobaltDataset
w = cobalt.Workspace(ds)

Let's open the UI and see how Cobalt helps understand the embeddings.

In [ ]:
w.ui.table_image_size = (160, 160)
w.ui

Here are the groups that we had manually selected above:

In [24]:
w.get_groups()

{'twin_group': CobaltDataSubset(source_dataset='Unnamed Dataset',
 columns=['targets', '_image_path'],
 media_columns=['_image_path'],
 timestamp_columns=[],
 hidable_columns=[],
 other_metadata_columns=[]
 )}

Before running the following cell, please make sure to create a group in the UI named 'twin_group' (or change the name of the group in the command below):

In [ ]:
# w.get_groups()['twin_group'].df

### Exploring the data in the UI

The different classes seem to be well separated in the graph shown above. You can double-click on nodes of the graph to select them, and open the data table to see the data contained in the selected node(s). 

You can also explore the automatically-generated clusters and see how well they align with the target classes. Each cluster seems to correspond to one class, but some classes are split into multiple clusters. See if you can come up with a hypothesis for why this might be.

It is also possible to access the analysis results and algorithms without using the UI.

Here are the groups that default settings of Cobalt found above:

In [26]:
clusters = w.clustering_results["auto_cluster"]
subgroups = [g.subset for g in clusters.groups]
len(subgroups)

15

Here `subgroups` is a list of all of the clusters that Cobalt found.

Let's look at the data in the first subgroup, as a `pd.DataFrame` by running: 

In [27]:
subgroups[0].df

,targets,_image_path
67,French horn,imagenette2/val/n03394916/n03394916_33711.JPEG
238,French horn,imagenette2/val/n03394916/n03394916_35731.JPEG
298,French horn,imagenette2/val/n03394916/n03394916_19491.JPEG
354,French horn,imagenette2/val/n03394916/n03394916_33380.JPEG
439,garbage truck,imagenette2/val/n03417042/n03417042_1271.JPEG
...,...,...
3716,cassette player,imagenette2/val/n02979186/n02979186_14462.JPEG
3741,cassette player,imagenette2/val/n02979186/n02979186_10402.JPEG
3764,cassette player,imagenette2/val/n02979186/n02979186_13740.JPEG
3857,cassette player,imagenette2/val/n02979186/n02979186_18940.JPEG


And you can see the images in it by running:

In [ ]:
w.view_table(subgroups[0])

Feel free to explore and try out more things!

More details about the UI can be found here: [https://docs.cobalt.bluelightai.com/ui.html](https://docs.cobalt.bluelightai.com/ui.html)

## Conclusion

- Cobalt gives a powerful way to interact and visualize complex data. 
- It is important to remember that the default settings used by Cobalt may not always perfectly separate classes, but it can provide a good starting point for further analysis. 
- The UI provides an easy way to explore and interact with the data, while also providing some basic visualizations.
    - While UI is very helpful in generating insights and interactive data analysis, it is not the only way to use Cobalt.
    - The API allows for more advanced usage and customization, such as automating workflows or integrating with other systems.
- Check out other videos to see how Cobalt can be used for other applications.

- Use Cobalt's API to automate more complex workflows or integrate it with other tools and systems.
- Download and use it today!

- And do let us know what you think about it:
<a href="https://bluelightai.com/contact">Give Feedback</a> | <a href="https://bluelightai.com/">Our Website</a> | <a href="https://bluelightai.com/blog">Our Blog</a> | <a href="https://docs.cobalt.bluelightai.com/">Cobalt Docs</a> | <a href="https://join.slack.com/t/bluelightaicom/shared_invite/zt-2uj0iu5lh-5WgutuwH82RxAOwuq8ptqg">Slack Community</a>

<div style="display: flex; align-items: center; justify-content: space-between;">
    <div style:"flex: 1; text-align: left;">
        <a href="#top" style="text-decoration: none; color: inherit;"> 
            <h3>Top of Page</h3> 
        </a>
    </div>
    <div style:"flex: 1; text-align: right;">
        <img style="width:50%;max-width:600px;float:right" alt="Bluelight AI Logo" href="https://bluelightai.com/" src="https://github.com/BlueLightAI/cobalt-examples/blob/main/assets/blai-logo-light.png?raw=true">
    </div>
</div>